In [1]:
import pandas as pd
import datetime as dt
import itertools
import os
# import matplotlib.pyplot as plt

### Variables
1. List of indices/forex to study
3. List of years & their ranges

In [81]:
data = dict.fromkeys('NSEI USDINR GBPINR JPYINR EURINR'.split(), None)

In [83]:
latest = dt.datetime(2019, 12, 31)
yrs = list(range(1, 11))
dates = {}
dates['range'] = {str(i) + ' yrs': (latest.replace(year=latest.year - i), latest) 
                  for i in yrs}
dates['years'] = {str(latest.year - i + 1): (latest.replace(year=latest.year - i), 
                                             latest.replace(year=latest.year -i + 1)) 
                  for i in yrs}

### Functions
1. To extract data from excel file
2. To return correlation table 

In [86]:
def calldata(name):
    filepath = f'./csv/{name}.csv'
    fileexist = os.path.isfile(filepath)
    if fileexist:
        return pd.read_csv(filepath, index_col=0, parse_dates=['Date'])
    else:
        if not os.path.isdir('./csv'): os.mkdir('./csv')
        readxl = pd.read_excel('FX_data.xlsx', sheet_name=name, index_col=0)
        readxl.to_csv(filepath)
        return readxl

In [55]:
def corr(df: 'merged df', value: 'column', rangemap: 'Dict'):
    df = df.xs(value, axis=1, level='Value')
    pairs = list(itertools.combinations(df.columns, 2))
    
    data = {}
    for (k, r) in rangemap.items():
        fdf = df[(df.index > r[0]) & (df.index <= r[1])] # filtered df
        data[k] = [fdf[a].corr(fdf[b]) for (a, b) in pairs]
    
    table = pd.DataFrame(data, index=pairs)
    table.columns.name = value
    return table

### Computations
Data is called from excel according to list of indices

In [75]:
for k in data:
    data[k] = calldata(k)

Data is merged in one single table

In [76]:
merged = pd.concat(data, axis=1, join='inner')
merged.columns.names = ['Symb', 'Value']

Extract of the merged data with 5970 rows

In [78]:
merged

Symb            NSEI                                                  USDINR  \
Value          Price      Open      High       Low     Vol. Change %   Price   
Date                                                                           
2019-12-31  12168.45  12247.10  12247.10  12151.80  426.93M  -0.0071  71.355   
2019-12-30  12255.85  12274.90  12286.45  12213.80  411.08M   0.0008  71.320   
2019-12-27  12245.80  12172.90  12258.45  12157.90  383.79M   0.0098  71.455   
2019-12-26  12126.55  12211.85  12221.55  12118.85  520.33M  -0.0072  71.300   
2019-12-24  12214.55  12269.25  12283.70  12202.10  470.29M  -0.0039  71.280   
...              ...       ...       ...       ...      ...      ...     ...   
1995-11-10    948.82    951.93    951.93    946.00        -  -0.0045  34.525   
1995-11-09    953.07    960.32    960.32    952.13        -  -0.0113  34.505   
1995-11-08    964.01    976.28    976.28    962.98        -  -0.0145  34.300   
1995-11-07    978.22    987.17    987.17    977.05        -  -0.0108  34.550   
1995-11-06    988.92   1001.53   1001.53    988.92        -  -0.0111  34.735   

Symb                                ...  JPYINR                          \
Value         Open    High     Low  ...   Price    Open    High     Low   
Date                                ...                                   
2019-12-31  71.275  71.385  71.225  ...  0.6567  0.6552  0.6580  0.6549   
2019-12-30  71.340  71.427  71.290  ...  0.6552  0.6524  0.6557  0.6521   
2019-12-27  71.305  71.505  71.185  ...  0.6521  0.6501  0.6535  0.6495   
2019-12-26  71.260  71.353  71.230  ...  0.6501  0.6519  0.6519  0.6501   
2019-12-24  71.210  71.317  71.153  ...  0.6517  0.6512  0.6521  0.6503   
...            ...     ...     ...  ...     ...     ...     ...     ...   
1995-11-10  34.285  34.635  34.285  ...  0.3423  0.3432  0.3462  0.3386   
1995-11-09  33.975  34.700  33.975  ...  0.3437  0.3338  0.3459  0.3312   
1995-11-08  34.340  34.822  34.300  ...  0.3344  0.3348  0.3382  0.3319   
1995-11-07  34.660  34.810  34.550  ...  0.3354  0.3358  0.3400  0.3339   
1995-11-06  34.185  34.865  34.185  ...  0.3361  0.3321  0.3389  0.3281   

Symb                  EURINR                                      
Value      Change %    Price     Open     High      Low Change %  
Date                                                              
2019-12-31   0.0023  80.0030  79.8590  80.2440  79.8100   0.0016  
2019-12-30   0.0043  79.8715  79.8115  80.0765  79.7465   0.0001  
2019-12-27   0.0031  79.8615  79.1235  80.0035  79.1165   0.0093  
2019-12-26  -0.0029  79.1285  79.0635  79.2215  78.9955   0.0009  
2019-12-24   0.0007  79.0350  78.9760  79.0490  78.8440   0.0008  
...             ...      ...      ...      ...      ...      ...  
1995-11-10  -0.0041  44.5980  44.5270  44.5980  44.5270  -0.0090  
1995-11-09   0.0278  45.0035  44.9450  45.0035  44.9450   0.0150  
1995-11-08  -0.0030  44.3395  44.2580  44.3395  44.2580  -0.0099  
1995-11-07  -0.0021  44.7840  44.6890  44.7840  44.6890  -0.0053  
1995-11-06   0.0072  45.0235  44.9610  45.0235  44.9610   0.0047  

[5970 rows x 26 columns]

### Correlations for defined ranges
1. Price
    - By range
    - By years
2. Change %
    - By range
    - By years

In [84]:
t = [(value, key) for value in ['Price', 'Change %'] for key in dates]

In [85]:
for (value, key) in t:
    print(f'Correlation of {value} for {key}:')
    display(corr(merged, value, dates[key]))

Correlation of Price for range:


Price,1 yrs,2 yrs,3 yrs,4 yrs,5 yrs,6 yrs,7 yrs,8 yrs,9 yrs,10 yrs
"(NSEI, USDINR)",-0.218937,0.318219,0.564541,0.409618,0.512608,0.661338,0.749470,0.822410,0.819530,0.829088
"(NSEI, GBPINR)",0.110817,-0.159875,0.640238,-0.020087,-0.341866,-0.550598,-0.353949,-0.060189,0.225855,0.375087
"(NSEI, JPYINR)",-0.213170,0.354279,0.677977,0.363851,0.557129,0.492703,0.335815,-0.053478,-0.006187,0.164059
"(NSEI, EURINR)",-0.483918,-0.407294,0.664850,0.658648,0.702840,0.187016,0.068755,0.336062,0.486193,0.569577
"(USDINR, GBPINR)",0.453267,0.365246,0.674357,0.460452,-0.000820,-0.365972,-0.073339,0.231326,0.580454,0.715818
"(USDINR, JPYINR)",0.808118,0.864847,0.913352,0.757788,0.805578,0.687516,0.533014,-0.000733,0.124994,0.352983
"(USDINR, EURINR)",0.666188,0.423172,0.635074,0.602278,0.714361,0.157300,0.206388,0.494233,0.694818,0.790019
"(GBPINR, JPYINR)",-0.060866,0.158552,0.668701,0.312117,-0.340874,-0.340513,-0.187442,-0.299699,-0.094011,0.214494
"(GBPINR, EURINR)",0.592899,0.764180,0.905077,0.561446,0.001963,0.292126,0.438671,0.556179,0.720786,0.816913
"(JPYINR, EURINR)",0.412927,0.231036,0.668604,0.532685,0.716664,0.550533,0.574893,0.143515,0.221202,0.432780


Correlation of Price for years:


Price,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
"(NSEI, USDINR)",-0.218937,0.245243,-0.781582,-0.421245,-0.892489,-0.039274,-0.031725,0.023460,-0.801049,-0.550910
"(NSEI, GBPINR)",0.110817,-0.156424,0.524272,-0.711649,-0.706269,-0.683987,0.153686,0.256973,-0.815051,0.132429
"(NSEI, JPYINR)",-0.213170,0.118519,-0.668540,0.751600,-0.774368,-0.791079,-0.200429,-0.224453,-0.873594,0.617507
"(NSEI, EURINR)",-0.483918,0.057085,0.752979,-0.150241,-0.658032,-0.939584,0.090251,0.332366,-0.754461,0.000047
"(USDINR, GBPINR)",0.453267,0.642323,-0.039786,0.024891,0.826016,-0.005613,0.954046,0.936778,0.959398,0.295771
"(USDINR, JPYINR)",0.808118,0.910745,0.588571,-0.308560,0.779126,-0.258135,0.794527,0.854570,0.956479,0.210180
"(USDINR, EURINR)",0.666188,0.794440,-0.327529,-0.225914,0.602878,-0.034979,0.975857,0.800012,0.816667,-0.146126
"(GBPINR, JPYINR)",-0.060866,0.739897,-0.289281,-0.439466,0.517617,0.856883,0.789171,0.721296,0.935315,0.490914
"(GBPINR, EURINR)",0.592899,0.921046,0.782247,0.578355,0.691564,0.842957,0.985661,0.916500,0.900364,0.671652
"(JPYINR, EURINR)",0.412927,0.855320,-0.327389,0.370137,0.667881,0.875606,0.792379,0.489229,0.864552,0.025327


Correlation of Change % for range:


Change %,1 yrs,2 yrs,3 yrs,4 yrs,5 yrs,6 yrs,7 yrs,8 yrs,9 yrs,10 yrs
"(NSEI, USDINR)",-0.272662,-0.326749,-0.314748,-0.322584,-0.342834,-0.342271,-0.387004,-0.398419,-0.408158,-0.423308
"(NSEI, GBPINR)",-0.157407,-0.165803,-0.145504,-0.083425,-0.124783,-0.144237,-0.230038,-0.242376,-0.239393,-0.242809
"(NSEI, JPYINR)",-0.202899,-0.219431,-0.199792,-0.306792,-0.353316,-0.359373,-0.388556,-0.399736,-0.393881,-0.392326
"(NSEI, EURINR)",-0.247938,-0.232410,-0.222665,-0.240749,-0.294778,-0.295037,-0.335657,-0.335236,-0.312152,-0.297255
"(USDINR, GBPINR)",0.488962,0.507221,0.455935,0.312864,0.344849,0.389393,0.540935,0.570076,0.565500,0.552104
"(USDINR, JPYINR)",0.745612,0.708727,0.588400,0.506317,0.532989,0.550956,0.610667,0.649130,0.651503,0.657858
"(USDINR, EURINR)",0.728447,0.652635,0.570515,0.502231,0.483590,0.517685,0.606035,0.608518,0.575920,0.540261
"(GBPINR, JPYINR)",0.409986,0.450569,0.422708,0.189624,0.258756,0.292358,0.423535,0.454204,0.452239,0.433601
"(GBPINR, EURINR)",0.625910,0.700475,0.652712,0.569678,0.583982,0.597928,0.661217,0.678746,0.677479,0.674292
"(JPYINR, EURINR)",0.663581,0.616826,0.577922,0.516329,0.542334,0.563406,0.584016,0.583731,0.558066,0.520213


Correlation of Change % for years:


Change %,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
"(NSEI, USDINR)",-0.272662,-0.379658,-0.251622,-0.359495,-0.405571,-0.344327,-0.508931,-0.464309,-0.466979,-0.545602
"(NSEI, GBPINR)",-0.157407,-0.175142,-0.092291,0.024068,-0.274142,-0.287126,-0.490719,-0.338080,-0.235886,-0.269441
"(NSEI, JPYINR)",-0.202899,-0.236741,-0.150933,-0.476235,-0.506068,-0.387253,-0.473061,-0.463668,-0.373137,-0.386279
"(NSEI, EURINR)",-0.247938,-0.220846,-0.208296,-0.286104,-0.406899,-0.291762,-0.462468,-0.335369,-0.207497,-0.192120
"(USDINR, GBPINR)",0.488962,0.526763,0.327399,-0.031787,0.480005,0.693559,0.849904,0.779561,0.527741,0.441033
"(USDINR, JPYINR)",0.745612,0.679959,0.260206,0.381136,0.632767,0.644408,0.719277,0.834687,0.663155,0.715537
"(USDINR, EURINR)",0.728447,0.596925,0.350196,0.285331,0.483801,0.704808,0.810950,0.644037,0.374703,0.281531
"(GBPINR, JPYINR)",0.409986,0.491844,0.365807,-0.146424,0.558000,0.536393,0.741854,0.679112,0.433483,0.307758
"(GBPINR, EURINR)",0.625910,0.769052,0.551425,0.403382,0.705610,0.712579,0.841595,0.816077,0.692239,0.655011
"(JPYINR, EURINR)",0.663581,0.581109,0.498557,0.428066,0.655764,0.684966,0.644078,0.585909,0.399075,0.284819
